## Face Recognition – Unlock Your Computer With Your Face!


### Step 1 - Create Training Data

In [2]:
import cv2
import numpy as np
# import pyfiglet module
import pyfiglet

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(1)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
result = pyfiglet.figlet_format("Samples Complete", font = "bulbhead" )
print(result)

<>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-2-daba3e553888>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


Face not found
Face not found
Face not found
Face not found
Face not found
 ___    __    __  __  ____  __    ____  ___ 
/ __)  /__\  (  \/  )(  _ \(  )  ( ___)/ __)
\__ \ /(__)\  )    (  )___/ )(__  )__) \__ \
(___/(__)(__)(_/\/\_)(__)  (____)(____)(___/
  ___  _____  __  __  ____  __    ____  ____  ____ 
 / __)(  _  )(  \/  )(  _ \(  )  ( ___)(_  _)( ___)
( (__  )(_)(  )    (  )___/ )(__  )__)   )(   )__) 
 \___)(_____)(_/\/\_)(__)  (____)(____) (__) (____)




### Step 2 - Train Model

In [3]:
cap.release()

In [4]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = './faces/user/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

vimal_model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
vimal_model.train(np.asarray(Training_Data), np.asarray(Labels))
result = pyfiglet.figlet_format("Model trained sucessefully", font = "bulbhead" )
print(result)

 __  __  _____  ____  ____  __   
(  \/  )(  _  )(  _ \( ___)(  )  
 )    (  )(_)(  )(_) ))__)  )(__ 
(_/\/\_)(_____)(____/(____)(____)
 ____  ____    __    ____  _  _  ____  ____  
(_  _)(  _ \  /__\  (_  _)( \( )( ___)(  _ \ 
  )(   )   / /(__)\  _)(_  )  (  )__)  )(_) )
 (__) (_)\_)(__)(__)(____)(_)\_)(____)(____/ 
 ___  __  __  ___  ____  ___  ___  ____  ____  __  __  __    __   _  _ 
/ __)(  )(  )/ __)( ___)/ __)/ __)( ___)( ___)(  )(  )(  )  (  ) ( \/ )
\__ \ )(__)(( (__  )__) \__ \\__ \ )__)  )__)  )(__)(  )(__  )(__ \  / 
(___/(______)\___)(____)(___/(___/(____)(__)  (______)(____)(____)(__) 



### Step 3 - Run Our Facial Recognition

In [6]:
import cv2
import numpy as np
import os
import time


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(1)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = vimal_model.predict(face)
        # harry_model.predict(face)
        
        if results[1] <500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is Swarnim'
            cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
       
            if confidence > 80:
                try:
                
                    cv2.putText(image, "Hey Swarnim", (255, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,100,0), 2)
                    cv2.imshow('Face Recognition', image )
                    cv2.putText(image, "Hey Swarnim", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                    cv2.imshow('Face Recognition', image )            
                    mail()
                    msg()
                    terraform()
                    break
                except:
                    print("Error")
            else:
                terraform()
                
    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()

<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-6-08183ed6fda7>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


 __  __  _____  ____  ____  __   
(  \/  )(  _  )(  _ \( ___)(  )  
 )    (  )(_)(  )(_) ))__)  )(__ 
(_/\/\_)(_____)(____/(____)(____)
 ____  ____    __    ____  _  _  ____  ____  
(_  _)(  _ \  /__\  (_  _)( \( )( ___)(  _ \ 
  )(   )   / /(__)\  _)(_  )  (  )__)  )(_) )
 (__) (_)\_)(__)(__)(____)(_)\_)(____)(____/ 
 ___  __  __  ___  ____  ___  ___  ____  ____  __  __  __    __   _  _ 
/ __)(  )(  )/ __)( ___)/ __)/ __)( ___)( ___)(  )(  )(  )  (  ) ( \/ )
\__ \ )(__)(( (__  )__) \__ \\__ \ )__)  )__)  )(__)(  )(__  )(__ \  / 
(___/(______)\___)(____)(___/(___/(____)(__)  (______)(____)(____)(__) 

 _    _  _   _    __   ____  ___    __    ____  ____    __  __  ___   ___ 
( \/\/ )( )_( )  /__\ (_  _)/ __)  /__\  (  _ \(  _ \  (  \/  )/ __) / __)
 )    (  ) _ (  /(__)\  )(  \__ \ /(__)\  )___/ )___/   )    ( \__ \( (_-.
(__/\__)(_) (_)(__)(__)(__) (___/(__)(__)(__)  (__)    (_/\/\_)(___/ \___/
 ___  ____  _  _  ____ 
/ __)( ___)( \( )(_  _)
\__ \ )__)  )  (   )(  
(___/(____)(_)\_) (

In [9]:
def mail():
    import smtplib
    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.starttls()
    s.login("technicistic@gmail.com", "*************")
    message = "Hello!! You are catched in camera!!!"
    s.sendmail("technicistic@gmail.com", "swarnim1245@gmail.com", message)
    s.quit()
    result = pyfiglet.figlet_format("Mail sent successfully", font = "bulbhead" )
    print(result)
def msg():
    import pywhatkit
    pywhatkit.sendwhatmsg_instantly("+919536750500", "Helloo!!, This is from Model msg successfully created...😍")
    result = pyfiglet.figlet_format("Whatsapp msg sent", font = "bulbhead" )
    print(result)
    
def terraform():
    import os
    os.system("terraform init") #downloading the plugins
    os.system("terraform apply --auto-approve")
    result = pyfiglet.figlet_format("Instance launch successfully", font = "bulbhead" )
    print(result)
       

In [10]:
mail()

SMTPAuthenticationError: (501, b'5.5.2 Cannot Decode response v9sm18815838pfn.22 - gsmtp')

In [ ]:
mail()

In [ ]:
terraform()

In [ ]:
# import pyfiglet module
import pyfiglet

result = pyfiglet.figlet_format("Mail sent successfully", font = "bulbhead" )
print(result)
